In [2]:
### pipenv install scikit-learn pandas plotly matplotlib pingouin nbformat ipykernel optuna ipywidgets gradio

# EDA
import pandas as pd
import numpy as np
import pingouin as pg
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt 

# ML
from sklearn.model_selection import cross_validate, StratifiedKFold, cross_val_predict, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Hyperparameter optimization
import optuna


In [3]:
# Carregar o dataset
df_segmento = pd.read_csv('./datasets/dataset_segmento_cliente.csv')

### EDA

In [4]:
# visualizar os dados 
df_segmento.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,segmento_de_cliente
0,Comércio,713109.95,12,Rio de Janeiro,6,1,Bronze
1,Comércio,790714.38,9,São Paulo,15,0,Bronze
2,Comércio,1197239.33,17,São Paulo,4,9,Silver
3,Indústria,449185.78,15,São Paulo,6,0,Starter
4,Agronegócio,1006373.16,15,São Paulo,15,8,Silver
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,Silver
6,Serviços,771179.95,13,Vitória,0,1,Starter
7,Serviços,707837.61,16,São Paulo,10,6,Silver
8,Comércio,888983.66,17,Belo Horizonte,10,1,Bronze
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,Bronze


In [5]:
# Estrutura do dataset
df_segmento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
 6   segmento_de_cliente     500 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 27.5+ KB


In [6]:
# valores possíveis - Variaveis categoricas
df_segmento['atividade_economica'].unique()

array(['Comércio', 'Indústria', 'Agronegócio', 'Serviços'], dtype=object)

In [7]:
# valores possíveis - Variaveis categoricas
df_segmento['localizacao'].unique()

array(['Rio de Janeiro', 'São Paulo', 'Vitória', 'Belo Horizonte'],
      dtype=object)

In [8]:
# valores possíveis - Variaveis categoricas
df_segmento['segmento_de_cliente'].unique()

array(['Bronze', 'Silver', 'Starter', 'Gold'], dtype=object)

In [9]:
# valores possíveis - Variaveis categoricas
df_segmento['inovacao'].unique()

array([1, 0, 9, 8, 4, 6, 3, 7, 5, 2])

In [10]:
# Distribuição da variavel segmento_de_cliente (target)
contagem_target = df_segmento['segmento_de_cliente'].value_counts()
contagem_target

segmento_de_cliente
Silver     260
Bronze     202
Starter     22
Gold        16
Name: count, dtype: int64

In [11]:
# Criar uma lista ordenada do target
lista_segmentos = ['Starter', 'Bronze', 'Silver', 'Gold']

In [12]:
# distribuição da variavel - Contagem
px.bar(contagem_target,color=contagem_target.index ,
       category_orders={'segmento_de_cliente': lista_segmentos}, 
       title='Distribuição da variável target')

In [13]:
# Distribuição da variavel Target - Percentual
percentual_target = contagem_target / len(df_segmento) * 100
px.bar(percentual_target,color=percentual_target.index ,
       category_orders={'segmento_de_cliente': lista_segmentos}, 
       title='Distribuição da variável target')

In [15]:
# Distribuição da variavel Localização
percentual_localizacao = df_segmento.value_counts('localizacao') / len(df_segmento) * 100
px.bar(percentual_localizacao,color=percentual_localizacao.index ,
       title='Distribuição da variável target')

In [16]:
# Distribuição da variavel Atividade economica
percentual_atividade = df_segmento.value_counts('atividade_economica') / len(df_segmento) * 100
px.bar(percentual_atividade,color=percentual_atividade.index ,
       title='Distribuição da variável target')

In [17]:
# Distribuição da variavel Inovação
percentual_inovacao = df_segmento.value_counts('inovacao') / len(df_segmento) * 100
px.bar(percentual_inovacao,color=percentual_inovacao.index ,
       title='Distribuição da variável target')

In [ ]:
# Tabela de contingência entre localização e Target (segmento de cliente)
crosstab_localizacao = pd.crosstab(df_segmento['localizacao'], 
                                   df_segmento['segmento_de_cliente'],
                                     margins=True)[lista_segmentos].reset_index()

tabela_localizacao = ff.create_table(crosstab_localizacao)
#mostrar a crosstab
tabela_localizacao.show()

In [24]:
# Tabela de contingência entre atividade economica e Target (segmento de cliente)
crosstab_atividade = pd.crosstab(df_segmento['atividade_economica'], 
                                 df_segmento['segmento_de_cliente'], 
                                 margins=True)[lista_segmentos].reset_index()

tabela_atividade = ff.create_table(crosstab_atividade)
#mostrar a crosstab
tabela_atividade.show()

In [25]:
# Tabela de contingência entre inovação e Target (segmento de cliente)
crosstab_inovacao = pd.crosstab(df_segmento['inovacao'], 
                                df_segmento['segmento_de_cliente'], 
                                margins=True)[lista_segmentos].reset_index()
tabela_atividade = ff.create_table(crosstab_inovacao)
#mostrar a crosstab
tabela_atividade.show()

In [28]:
# distribuição idade da empresa
px.histogram(df_segmento, x='idade', color='segmento_de_cliente', 
             title='Distribuição da variável idade_empresa')

In [29]:
# distribuição faturamento
px.histogram(df_segmento, x='faturamento_mensal', color='segmento_de_cliente', 
             title='Distribuição da variável faturamento')

In [30]:
# Verificar a correlação entre as variáveis
# Boxplot entre idade e segmento de cliente
px.box(df_segmento, x='segmento_de_cliente', y='idade', 
       title='Boxplot da variável idade por segmento de cliente',
       color='segmento_de_cliente',
       category_orders={'segmento_de_cliente': lista_segmentos})

In [31]:
# Boxplot entre faturamento e segmento de cliente
px.box(df_segmento, x='segmento_de_cliente', y='faturamento_mensal', 
       title='Boxplot da variável faturamento por segmento de cliente',
       color='segmento_de_cliente',
       category_orders={'segmento_de_cliente': lista_segmentos})

In [32]:
# Correlação variáveis qualitativas
# Teste de qui-quadrado de pearson - Teste de independência
# Teste de hipótese
# H0: As variáveis são independentes
# H1: As variáveis são dependentes (não são independentes)
# se o p-valor for menor que 0.05 rejeitamos a hipótese nula
# se o p-valor for maior que 0.05 não rejeitamos a hipótese nula

# Teste de independência entre segmento de cliente e localização
valor_esperado, valor_observado, estatísticas =  pg.chi2_independence(df_segmento, x='segmento_de_cliente', y='localizacao')

/home/srmatheusmaciel/.local/share/virtualenvs/classificacao_arvore_decisao-pApQBjQT/lib/python3.12/site-packages/pingouin/contingency.py:151: UserWarning:

Low count on observed frequencies.

/home/srmatheusmaciel/.local/share/virtualenvs/classificacao_arvore_decisao-pApQBjQT/lib/python3.12/site-packages/pingouin/contingency.py:151: UserWarning:

Low count on expected frequencies.



In [33]:
# Valor esperado
# É a frequencia que seria esperada se não houvesse relação entre as variáveis
# é calculado utilizando a distribuição assumida no test do qui-quadrado
valor_esperado

localizacao,Belo Horizonte,Rio de Janeiro,São Paulo,Vitória
segmento_de_cliente,,,,
Bronze,44.844,52.924,48.884,55.348
Gold,3.552,4.192,3.872,4.384
Silver,57.720,68.120,62.920,71.240
Starter,4.884,5.764,5.324,6.028


In [34]:
# Valor observado
# É a frequencia real observada entre as variáveis
valor_observado

localizacao,Belo Horizonte,Rio de Janeiro,São Paulo,Vitória
segmento_de_cliente,,,,
Bronze,39,62,45,56
Gold,4,3,5,4
Silver,63,60,65,72
Starter,5,6,6,5


In [35]:
# Estatísticas
# Estatísticas do teste do qui-quadrado
estatísticas.round(2)

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.00,5.19,9.0,0.82,0.06,0.11
1,cressie-read,0.67,5.19,9.0,0.82,0.06,0.11
2,log-likelihood,0.00,5.20,9.0,0.82,0.06,0.11
3,freeman-tukey,-0.50,5.21,9.0,0.82,0.06,0.11
4,mod-log-likelihood,-1.00,5.22,9.0,0.81,0.06,0.11
5,neyman,-2.00,5.28,9.0,0.81,0.06,0.11


As variaveis localização e segmento de cliente são independentes.
Qui-Quadrado (p-value = 0.82)

In [36]:
# Teste de independência entre segmento de cliente e atividade economica
valor_esperado, valor_observado, estatísticas =  pg.chi2_independence(df_segmento, x='segmento_de_cliente', y='atividade_economica')

/home/srmatheusmaciel/.local/share/virtualenvs/classificacao_arvore_decisao-pApQBjQT/lib/python3.12/site-packages/pingouin/contingency.py:151: UserWarning:

Low count on observed frequencies.

/home/srmatheusmaciel/.local/share/virtualenvs/classificacao_arvore_decisao-pApQBjQT/lib/python3.12/site-packages/pingouin/contingency.py:151: UserWarning:

Low count on expected frequencies.

/home/srmatheusmaciel/.local/share/virtualenvs/classificacao_arvore_decisao-pApQBjQT/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7344: RuntimeWarning:

divide by zero encountered in power

/home/srmatheusmaciel/.local/share/virtualenvs/classificacao_arvore_decisao-pApQBjQT/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7344: RuntimeWarning:

invalid value encountered in multiply

/home/srmatheusmaciel/.local/share/virtualenvs/classificacao_arvore_decisao-pApQBjQT/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7341: RuntimeWarning:

divide by zero encountered in divide



In [37]:
valor_esperado

atividade_economica,Agronegócio,Comércio,Indústria,Serviços
segmento_de_cliente,,,,
Bronze,47.672,56.156,49.288,48.884
Gold,3.776,4.448,3.904,3.872
Silver,61.360,72.280,63.440,62.920
Starter,5.192,6.116,5.368,5.324


In [38]:
valor_observado

atividade_economica,Agronegócio,Comércio,Indústria,Serviços
segmento_de_cliente,,,,
Bronze,47,56,50,49
Gold,7,5,4,0
Silver,57,75,63,65
Starter,7,3,5,7


In [39]:
estatísticas.round(2)

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.00,9.97,9.0,0.35,0.08,0.19
1,cressie-read,0.67,10.61,9.0,0.30,0.08,0.20
2,log-likelihood,0.00,13.55,9.0,0.14,0.10,0.25
3,freeman-tukey,-0.50,NaN,9.0,NaN,NaN,NaN
4,mod-log-likelihood,-1.00,inf,9.0,0.00,inf,NaN
5,neyman,-2.00,NaN,9.0,NaN,NaN,NaN


As variáveis atividade economica e segmento de cliente são independentes
Qui-Quadrado(p-value=0.35)

In [40]:
# Teste de independência entre segmento de cliente e inovação
valor_esperado, valor_observado, estatísticas =  pg.chi2_independence(df_segmento, x='segmento_de_cliente', y='inovacao')

/home/srmatheusmaciel/.local/share/virtualenvs/classificacao_arvore_decisao-pApQBjQT/lib/python3.12/site-packages/pingouin/contingency.py:151: UserWarning:

Low count on observed frequencies.

/home/srmatheusmaciel/.local/share/virtualenvs/classificacao_arvore_decisao-pApQBjQT/lib/python3.12/site-packages/pingouin/contingency.py:151: UserWarning:

Low count on expected frequencies.

/home/srmatheusmaciel/.local/share/virtualenvs/classificacao_arvore_decisao-pApQBjQT/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7344: RuntimeWarning:

divide by zero encountered in power

/home/srmatheusmaciel/.local/share/virtualenvs/classificacao_arvore_decisao-pApQBjQT/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7344: RuntimeWarning:

invalid value encountered in multiply

/home/srmatheusmaciel/.local/share/virtualenvs/classificacao_arvore_decisao-pApQBjQT/lib/python3.12/site-packages/scipy/stats/_stats_py.py:7341: RuntimeWarning:

divide by zero encountered in divide



In [41]:
valor_esperado

inovacao,0,1,2,3,4,5,6,7,8,9
segmento_de_cliente,,,,,,,,,,
Bronze,21.008,23.028,22.624,19.392,17.372,17.372,21.816,21.412,18.988,18.988
Gold,1.664,1.824,1.792,1.536,1.376,1.376,1.728,1.696,1.504,1.504
Silver,27.040,29.640,29.120,24.960,22.360,22.360,28.080,27.560,24.440,24.440
Starter,2.288,2.508,2.464,2.112,1.892,1.892,2.376,2.332,2.068,2.068


In [42]:
valor_observado

inovacao,0,1,2,3,4,5,6,7,8,9
segmento_de_cliente,,,,,,,,,,
Bronze,36,44,32,22,12,14,15,12,9,6
Gold,0,0,0,0,0,3,0,5,4,4
Silver,10,5,20,25,30,25,38,36,34,37
Starter,6,8,4,1,1,1,1,0,0,0


In [45]:
estatísticas.round(5)

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.00000,164.29399,27.0,0.0,0.33095,0.99850
1,cressie-read,0.66667,165.49946,27.0,0.0,0.33216,0.99861
2,log-likelihood,0.00000,181.48878,27.0,0.0,0.34784,0.99951
3,freeman-tukey,-0.50000,NaN,27.0,NaN,NaN,NaN
4,mod-log-likelihood,-1.00000,inf,27.0,0.0,inf,NaN
5,neyman,-2.00000,NaN,27.0,NaN,NaN,NaN


No caso da variavel inovação, há uma correlação entre a variavel target. Portanto as variaveis não são independentes. Qui-quadrado (p-value=0.0000)